<a href="https://colab.research.google.com/github/jeromepatel/DL-University-Course-Practicals/blob/master/18BCE088_DL_Practical_6_RNN_Text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical 6 - Text Generation using RNN

## Importing Libraries

In [123]:

#importing libraries
from matplotlib import pyplot
# from keras.datasets.cifar10 import load_data
from tensorflow import keras
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from keras.models import Sequential
from keras.layers import Dropout, Flatten, SimpleRNN
from keras import backend as K
# from keras.layers import Conv2D, MaxPool2D, BatchNormalization ,GlobalAveragePooling2D
#from keras.applications.inception_v3 import InceptionV3, preprocess_input
# from tensorflow.keras.applications.xception import Xception, preprocess_input
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from __future__ import print_function
import spacy
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU
import numpy
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from nltk import word_tokenize, sent_tokenize

In [124]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [125]:
#@title Alternative Text Data { display-mode: "form" }
data = """ The name of my village is Pakdiyar which falls in the Gopalganj district of Bihar . \n During the summer and winter vacations, I visit my village . \n My grandparents live in the village . \n They like the village more than the city where we stay . \n My grandparents’ house is one of the biggest pakka houses in the village . \n My grandfather is the sarpanch of the village . \n He is admired by all the villagers for his just verdicts and actions towards the welfare of the villagers . \n In my village, our family is highly respected . \n My grandmother does a lot of social work for the villagers ."""

In [126]:
corpus = """ Jack and Jill went up the hill . \n To fetch a pail of water . \n Jack fell down and broke his crown . \n And Jill came tumbling after ."""
tokenized_sent = corpus.split(' \n')

# tokenized_sent = sent_tokenize(data)
# tokenized_sent = data.split(' \n')
tokenized_sent

[' Jack and Jill went up the hill .',
 ' To fetch a pail of water .',
 ' Jack fell down and broke his crown .',
 ' And Jill came tumbling after .']

In [127]:
tokenizer=Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\]^_{/}~')

tokenizer.fit_on_texts(tokenized_sent)
# tokenizer.word_index
VOCAB_SIZE = len(tokenizer.word_index) + 1

In [128]:
VOCAB_SIZE

23

In [129]:
sequence = tokenizer.texts_to_sequences(tokenized_sent)
print(f"first sequence encoded: {sequence[0]}")

first sequence encoded: [3, 2, 4, 5, 6, 7, 8, 1]


In [130]:
X = []
Y = []
maxlen = 0
for i in range(len(sequence)):
    X.insert(i, sequence[i][:-1])
    Y.insert(i, sequence[i])
    maxlen = max(len(Y[i]),maxlen)

X[0], Y[0], maxlen

([3, 2, 4, 5, 6, 7, 8], [3, 2, 4, 5, 6, 7, 8, 1], 8)

In [131]:
X = pad_sequences(X,maxlen = maxlen+1, padding = 'pre')
Y = pad_sequences(Y, maxlen = maxlen+1, padding = 'pre')

X[0],Y[0]

(array([0, 0, 3, 2, 4, 5, 6, 7, 8], dtype=int32),
 array([0, 3, 2, 4, 5, 6, 7, 8, 1], dtype=int32))

In [132]:
Y = to_categorical(Y,VOCAB_SIZE)
Y[0]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [133]:
model=Sequential()
model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=40))
model.add(SimpleRNN(units=70, return_sequences=True))
model.add(Dense(units=VOCAB_SIZE, activation='softmax'))

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 40)          920       
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, None, 70)          7770      
_________________________________________________________________
dense_7 (Dense)              (None, None, 23)          1633      
Total params: 10,323
Trainable params: 10,323
Non-trainable params: 0
_________________________________________________________________


In [134]:
model.compile(optimizer='rmsprop',loss = 'categorical_crossentropy',metrics=['accuracy'])
model.fit(X,Y,epochs=200,verbose=0)

In [141]:
def probability_of_input_sentence(model, tokenizer, sentence):
    encoded = tokenizer.texts_to_sequences([sentence])[0]
    encoded.insert(0, 0)
    encoded = numpy.array(encoded)
    encoded = encoded.reshape(1, -1)
    prob = model.predict(encoded, verbose=0)
    probability = 1
    print(prob.shape)
    for i in range(prob.shape[1]-1):
        probability = probability * prob[0, i, encoded[0, i+1]]
    print(f"Probability of Sentence '{sentence}' is:", probability)

prob_of_input_sentence(model, tokenizer, " Jill and Jack will go down")

(1, 5, 23)
Probability of Sentence ' Jill and Jack will go down' is: 2.261855239839254e-13


## Text Generation (using argmax)
Here we generate text using predicting most probable word from output probabilities, this method can give more accurate answers than sampling.

In [136]:
def generate_sentence(model, tokenizer, n_words, vocab_size):
    sequence = [[0]]
    for i in range(n_words):
        probabilies = model.predict(sequence, verbose=0)
        temp = numpy.argmax(probabilies[0][-1])
        sequence[0] += [temp]
    
    sentence = tokenizer.sequences_to_texts(sequence)[0]
    return sentence

print(generate_sentence(model, tokenizer, 8, VOCAB_SIZE))

to fetch a pail of water


## Text Generation using Sampling Predictions
Here we use sampled probabilites as an input to model for prediction, this method works can be less accurate to generate text but provides more variation in text generation.

In [137]:
def sample_sentences(model, tokenizer, num_words,vocab_size):
  encoded = []
  text = ''
  for i in range(num_words):
    encoded = tokenizer.texts_to_sequences([text])[0]
    encoded.insert(0,0)
    encoded = numpy.array(encoded)
    encoded = numpy.reshape(encoded,newshape=(1,-1))

    print("\nFor:",i,"Encoded:",encoded,encoded.shape)
    if i == 0:
      probability = model.predict(encoded, verbose = 1)
    #   print("\nFor:",i,"Probability:",probability,type(probability),probability.shape)
      yhat = 0
      while(yhat == 0):
        yhat = numpy.random.choice(range(vocab_size),p=probability.ravel())
      yhat = [yhat]
      yhat = numpy.array(yhat)
      yhat = numpy.reshape(yhat,newshape = (1,-1))
    else:
      probability = model.predict(encoded,verbose=1)
    #   print("\nFor:",i,"Probability:",probability,type(probability),probability.shape)
      yhat = numpy.append(yhat,0)
      yhat = numpy.reshape(yhat,newshape=(1,-1))
      while yhat[0,i] == 0:
        yhat[0,i] = numpy.random.choice(range(vocab_size),p=probability[0,i].ravel())
    print("\nFor:",i,"Else yhat:",yhat,yhat.shape)

    out_word = ''
    for word,index in tokenizer.word_index.items():
      if index == yhat[0,i]:
        out_word = word
        break

    text = text + out_word + ' '
  return text
sample_sentences(model, tokenizer, 8, VOCAB_SIZE)


For: 0 Encoded: [[0]] (1, 1)
1/1 [==============================] - 0s 16ms/step

For: 0 Else yhat: [[21]] (1, 1)

For: 1 Encoded: [[ 0 21]] (1, 2)
1/1 [==============================] - 0s 15ms/step

For: 1 Else yhat: [[21  3]] (1, 2)

For: 2 Encoded: [[ 0 21  3]] (1, 3)
1/1 [==============================] - 0s 21ms/step

For: 2 Else yhat: [[21  3 15]] (1, 3)

For: 3 Encoded: [[ 0 21  3 15]] (1, 4)
1/1 [==============================] - 0s 14ms/step

For: 3 Else yhat: [[21  3 15 16]] (1, 4)

For: 4 Encoded: [[ 0 21  3 15 16]] (1, 5)
1/1 [==============================] - 0s 15ms/step

For: 4 Else yhat: [[21  3 15 16  2]] (1, 5)

For: 5 Encoded: [[ 0 21  3 15 16  2]] (1, 6)
1/1 [==============================] - 0s 14ms/step

For: 5 Else yhat: [[21  3 15 16  2 17]] (1, 6)

For: 6 Encoded: [[ 0 21  3 15 16  2 17]] (1, 7)
1/1 [==============================] - 0s 15ms/step

For: 6 Else yhat: [[21  3 15 16  2 17 18]] (1, 7)

For: 7 Encoded: [[ 0 21  3 15 16  2 17 18]] (1, 8)
1/1 [======

'tumbling jack fell down and broke his crown '

## Code by: 18BCE088 Jyot Makadiya
